In [1]:
import urllib.request, json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import json
import time

In [2]:
# import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument("--csv_file_name", required=True, help="Filename of the Reddit-scrape-data CSV we want author data for")
# parser.add_argument("--output", required=False, default='data/authorsubs.json', help="Filename to save to")
# args = parser.parse_args()
# print(' reading from:', args.csv_file_name)
# print('outputting to:', args.output)

# df_posts = pd.read_csv(args.csv_file_name)

In [4]:
def getAllType(username, contentType):
    n_posts = 500
    with urllib.request.urlopen(f"https://api.pushshift.io/reddit/search/{contentType}/?author={username}&sort=asc&size={n_posts}") as url:
        data = json.loads(url.read().decode())
        data = data['data']
    df_content = pd.DataFrame.from_dict(json_normalize(data), orient='columns')
    if len(df_content) == 0:
        return df_content
    created_utc_last = df_content.tail(1)['created_utc'].copy().reset_index()
    created_utc_last = created_utc_last['created_utc'][0]
    while len(data) > 0:
        with urllib.request.urlopen(f"https://api.pushshift.io/reddit/search/{contentType}/?author={username}&sort=asc&size={n_posts}&after={created_utc_last}") as url:
            data = json.loads(url.read().decode())
            data = data['data']
        df_content = df_content.append(pd.DataFrame.from_dict(json_normalize(data), orient='columns'))
        created_utc_last = df_content.tail(1)['created_utc'].copy().reset_index()
        created_utc_last = created_utc_last['created_utc'][0]
    return df_content

In [27]:
version = '2200'
df_posts = pd.read_csv(f'data/data_sample_{version}.csv')

In [28]:
output = f'data/authorsubs_{version}.json'

# Build subreddit mappings
with open(output, 'r') as fp:
    sub_mappings = json.load(fp)

print(len(sub_mappings))

2066


In [29]:
print(len(set(df_posts['author'])))

2169


In [ ]:
for username in set(df_posts['author']):
    with open(args.output, 'r') as fp:
        sub_mappings = json.load(fp)
    try:
        if username not in sub_mappings:
            df_comment = getAllType(username, 'comment').reset_index()
            df_submission = getAllType(username, 'submission').reset_index()
            df_comment_set = list(set(df_comment['subreddit'])) if len(df_comment) > 0 else []
            df_submission_set = list(set(df_submission['subreddit'])) if len(df_submission) > 0 else []
            sub_mappings[username] = {
                'comment': df_comment_set,
                'submission': df_submission_set
            }
    except:
        print('failed to read', username)
    finally:
        # save what we have so far if the last read attempt failed
        with open(args.output, 'w') as fp:
            json.dump(sub_mappings, fp)
        
#     print('got subreddits for', username)

print('PROCESS TIME ELAPSED (s)', time.process_time() - t0)

with open(args.output, 'w') as fp:
    json.dump(sub_mappings, fp)

In [ ]:
print(set(df_submission['subreddit']))

## Try with PRAW

In [32]:
import praw

reddit = praw.Reddit(client_id='Gq7arWbRvOEjRA',
                     client_secret='gyOeWVPNJUGuynTbHWPqQ3xJIJw',
                     user_agent='testscript by /u/AurigaCorvus',
                     username='AurigaCorvus',
                     password='dachytilioso391')

In [54]:
from praw.models import User, ListingGenerator

# print(reddit.user.me())
person = reddit.redditor('AurigaCorvus')
contribs = User.karma(person)

In [59]:
print(contribs)

{Subreddit(display_name='LifeProTips'): {'comment_karma': 139, 'link_karma': 1}, Subreddit(display_name='AsianParentStories'): {'comment_karma': 23, 'link_karma': 51}, Subreddit(display_name='fountainpens'): {'comment_karma': 8, 'link_karma': 50}, Subreddit(display_name='gradadmissions'): {'comment_karma': 2, 'link_karma': 41}, Subreddit(display_name='asexuality'): {'comment_karma': 27, 'link_karma': 1}, Subreddit(display_name='TwoXChromosomes'): {'comment_karma': 21, 'link_karma': 1}, Subreddit(display_name='discordapp'): {'comment_karma': 19, 'link_karma': 1}, Subreddit(display_name='traaaaaaannnnnnnnnns'): {'comment_karma': 10, 'link_karma': 1}, Subreddit(display_name='forbiddensnacks'): {'comment_karma': 0, 'link_karma': 8}, Subreddit(display_name='vivaldibrowser'): {'comment_karma': 5, 'link_karma': 3}, Subreddit(display_name='whatisthisthing'): {'comment_karma': 3, 'link_karma': 2}, Subreddit(display_name='demisexuality'): {'comment_karma': 5, 'link_karma': 1}, Subreddit(display_

In [60]:
for k in contribs.keys():
    print(k.display_name)
    if contribs[k]['comment_karma'] is not 0:
        print(k.display_name)
#     print(k.display_name)

LifeProTips
AsianParentStories
fountainpens
gradadmissions
asexuality
TwoXChromosomes
discordapp
traaaaaaannnnnnnnnns
vivaldibrowser
whatisthisthing
demisexuality
agender
aaaaaaacccccccce
SeattleWA
AskReddit
udub
AskAcademia
krita
Warframe
